In [17]:
import geopandas as gpd
import rasterio
from osgeo import gdal
import fiona
import pyproj

In [51]:
gdb_path = "/Users/propoop/Downloads/fgdb_env_cva_lst2022/env_cva_lst2022.gdb"

layers = fiona.listlayers(gdb_path)
print(layers)

['LandSurfaceTemperature2022', 'fras_aux_LandSurfaceTemperature2022', 'fras_blk_LandSurfaceTemperature2022', 'fras_bnd_LandSurfaceTemperature2022', 'fras_ras_LandSurfaceTemperature2022']


In [57]:
layer_name = "LandSurfaceTemperature2022"

gdf = gpd.read_file(gdb_path, layer=layer_name)

gdf.head()

/Users/propoop/miniconda3/envs/greenalysis/lib/python3.11/site-packages/geopandas/array.py:334: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  self.crs = crs
/Users/propoop/miniconda3/envs/greenalysis/lib/python3.11/site-packages/geopandas/geodataframe.py:409: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  level.crs = crs


,RASTER,FOOTPRINT_Length,FOOTPRINT_Area,geometry
0,1,423240.0,1.117542e+10,"MULTIPOLYGON (((419960 4919430, 419960 5029750..."


In [62]:
import fiona
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
import numpy as np

# Path to your File Geodatabase
gdb_path = '/Users/propoop/Downloads/fgdb_env_cva_lst2022/env_cva_lst2022.gdb'

# The name of the raster layer you want to read
layer_to_read = 'LandSurfaceTemperature2022'

try:
    print(f"Attempting to open GDB: {gdb_path}")
    print(f"Attempting to read layer: {layer_to_read}")

    # --- THE FIX IS HERE ---
    # Pass the GDB path and the layer name as separate arguments.
    with rasterio.open(gdb_path, layer=layer_to_read) as src:
        print("\nSuccessfully opened raster layer!")
        print(f"  - CRS: {src.crs}")
        print(f"  - Dimensions: {src.width}x{src.height}")
        
        # Read the first band of the raster into a NumPy array
        temp_data = src.read(1)
        
        # Get the NoData value and mask the array
        nodata_value = src.nodata
        if nodata_value is not None:
            print(f"  - NoData value found: {nodata_value}. Masking data.")
            temp_data = np.where(temp_data == nodata_value, np.nan, temp_data)

    # --- Plotting the temperature data (same as before) ---
    print("\nPlotting the data...")
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    im = ax.imshow(temp_data, cmap='inferno')
    
    ax.set_title('Land Surface Temperature 2022')
    ax.set_xlabel('X Coordinate (pixels)')
    ax.set_ylabel('Y Coordinate (pixels)')

    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Temperature')

    plt.show()

except rasterio.errors.RasterioIOError as e:
    # This error will now be more informative if it still fails
    print(f"Rasterio Error: {e}. Check that the layer name is correct and the GDB is not corrupt.")
    print("\nIf this fails, see Solution 2 below.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Attempting to open GDB: /Users/propoop/Downloads/fgdb_env_cva_lst2022/env_cva_lst2022.gdb
Attempting to read layer: LandSurfaceTemperature2022
Rasterio Error: '/Users/propoop/Downloads/fgdb_env_cva_lst2022/env_cva_lst2022.gdb' not recognized as a supported file format.. Check that the layer name is correct and the GDB is not corrupt.

If this fails, see Solution 2 below.


In [66]:
from osgeo import gdal
print(gdal.VersionInfo())
print(gdal.GetDriverByName('OpenFileGDB'))


3060200
<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x166e7fae0> >


In [70]:
gdb_ds = gdal.OpenEx(gdb_path, gdal.OF_READONLY | gdal.OF_RASTER)
if gdb_ds is None:
    print("❌ Could not open GDB.")
else:
    subdatasets = gdb_ds.GetSubDatasets()
    if not subdatasets:
        print("❌ No subdatasets (layers) found.")
    else:
        print("✅ Found subdatasets:")
        for i, sd in enumerate(subdatasets):
            print(f"{i+1}: {sd[0]}")


❌ Could not open GDB.


In [71]:
from osgeo import gdal

print("GDAL version:", gdal.__version__)
print("\nDrivers:")

for i in range(gdal.GetDriverCount()):
    driver = gdal.GetDriver(i)
    print(driver.ShortName)


GDAL version: 3.6.2

Drivers:
VRT
DERIVED
GTiff
COG
NITF
RPFTOC
ECRGTOC
HFA
SAR_CEOS
CEOS
JAXAPALSAR
GFF
ELAS
ESRIC
AIG
AAIGrid
GRASSASCIIGrid
ISG
SDTS
DTED
PNG
JPEG
MEM
JDEM
GIF
BIGGIF
ESAT
FITS
BSB
XPM
BMP
DIMAP
AirSAR
RS2
SAFE
PCIDSK
PCRaster
ILWIS
SGI
SRTMHGT
Leveller
Terragen
netCDF
HDF4
HDF4Image
ISIS3
ISIS2
PDS
PDS4
VICAR
TIL
ERS
JP2OpenJPEG
L1B
FIT
GRIB
RMF
WCS
WMS
MSGN
RST
GSAG
GSBG
GS7BG
COSAR
TSX
COASP
R
MAP
KMLSUPEROVERLAY
WEBP
PDF
Rasterlite
MBTiles
PLMOSAIC
CALS
WMTS
SENTINEL2
MRF
TileDB
PNM
DOQ1
DOQ2
PAux
MFF
MFF2
GSC
FAST
BT
LAN
CPG
NDF
EIR
DIPEx
LCP
GTX
LOSLAS
NTv2
CTable2
ACE2
SNODAS
KRO
ROI_PAC
RRASTER
BYN
ARG
RIK
USGSDEM
GXF
KEA
BAG
HDF5
HDF5Image
NWT_GRD
NWT_GRC
ADRG
SRP
BLX
PostGISRaster
SAGA
XYZ
HF2
OZI
CTG
ZMap
NGSGEOID
IRIS
PRF
EEDAI
EEDA
DAAS
SIGDEM
TGA
OGCAPI
STACTA
STACIT
GNMFile
GNMDatabase
ESRI Shapefile
MapInfo File
UK .NTF
LVBAG
OGR_SDTS
S57
DGN
OGR_VRT
Memory
CSV
NAS
GML
GPX
LIBKML
KML
GeoJSON
GeoJSONSeq
ESRIJSON
TopoJSON
Interlis 1
Interlis 2
OGR_GMT
G

In [72]:
from osgeo import gdal

gdb_path = "/path/to/your/LST2022.gdb"
dataset = gdal.OpenEx(gdb_path, gdal.OF_READONLY | gdal.OF_RASTER)

if dataset is None:
    print("❌ Could not open GDB dataset.")
else:
    subdatasets = dataset.GetSubDatasets()
    if not subdatasets:
        print("❌ No subdatasets (layers) found.")
    else:
        print("✅ Found subdatasets:")
        for i, (name, desc) in enumerate(subdatasets):
            print(f"{i}: {desc}\n{name}")


❌ Could not open GDB dataset.


In [73]:
from osgeo import gdal
print(gdal.GetDriverCount())
print(gdal.GetDriverByName('OpenFileGDB'))
print(gdal.GetDriverByName('FileGDB'))  # Only works with ESRI's FileGDB API


205
<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x16945a5b0> >
None
